Import are packages.

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np 
from shapely.geometry import Point,Polygon
import descartes
import statsmodels
from statsmodels.formula.api import ols

Lets load are clean data file as a pandas data frame.

In [2]:
df = pd.read_csv('data/kc_house_data_clean')

And doulbe check to make sure that all the info we are wanting to use is a int64 or float64.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     21597 non-null  float64
 9   view           21597 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  float64
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   21597 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

# Models

### First Simple Model

Are first simple model (FSM) is going to be are target: price and are first parameter: sqft_living.

In [4]:
first_formula = 'price ~ sqft_living'
model = ols(first_formula,df).fit()
model_summary = model.summary()
model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                 2.097e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        13:44:56   Log-Likelihood:            -3.0006e+05
No. Observations:               21597   AIC:                         6.001e+05
Df Residuals:                   21595   BIC:                         6.001e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -4.399e+04   4410.023     -9.975      0.000   -5.26e+04   -3.53e+04
sqft_living   280.8630      1.939    144.819      0.000     277.062     284.664
==============================================================================
Omnibus:                    14801.942   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           542662.604
Skew:                           2.820   Prob(JB):                         0.00
Kurtosis:                      26.901   Cond. No.                     5.63e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.63e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Cool are first FSM has a R-squared of: 0.493 not high or low but it is a start. Lets start adding in other parameters and see if we can improve that any.

Before we go on lets make a function or to for making models that we can use.

### Functions

Here we have a function for making models.

In [5]:
def model_tests(model_formula, df):
    formula = 'price ~ sqft_living + ' + model_formula
    model = ols(formula, df).fit()
    model_summary = model.summary()
    return model_summary

Now there is sure to be some columns that are categorical so lets make a function for that as well.

In [6]:
def dummy_model(column, df):
    dummy = pd.get_dummies(df[column], prefix=column, drop_first=True)
    df_dummy = pd.concat([df, dummy], axis=1)
    
    dummy_formula = ''
    dummy_formula = ' + '.join(df_dummy.columns[-((len(df[column].value_counts())-1)):])

    formula = 'price ~ sqft_living + ' + dummy_formula
    model = ols(formula, df_dummy).fit()
    model_summary = model.summary()
    return model_summary

Are last function will be for making a dataframe for storing all are R-squeared values so we can look are them all in one area.

In [7]:
R_2_dict = {}

def add_R_2(model_name, r_2):
    R_2_dict[model_name] = r_2
    df_r_2 = pd.DataFrame(list(R_2_dict.items()), columns = ['Name', 'R-squared'])
    return df_r_2

Let store are FSM R-squared in are fucntion first before moving on.

In [8]:
add_R_2('FSM', 0.493)

,Name,R-squared
0,FSM,0.493


### Grade Model

Starting with the grade column, let start by checking out what is in the this comlumn.

In [9]:
df['grade'].value_counts()

7     8974
8     6065
9     2615
6     2038
10    1134
11     399
5      242
12      89
4       27
13      13
3        1
Name: grade, dtype: int64

Looks like this may be a categorical column so let try using it as is and then all so use a 2nd from with dummies and see what happens.

In [10]:
model_tests('grade',df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.535
Model:                            OLS   Adj. R-squared:                  0.535
Method:                 Least Squares   F-statistic:                 1.241e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        13:44:56   Log-Likelihood:            -2.9913e+05
No. Observations:               21597   AIC:                         5.983e+05
Df Residuals:                   21594   BIC:                         5.983e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -6.028e+05   1.33e+04    -45.181      0.000   -6.29e+05   -5.77e+05
sqft_living   184.1215      2.872     64.102      0.000     178.491     189.751
grade        9.925e+04   2247.784     44.155      0.000    9.48e+04    1.04e+05
==============================================================================
Omnibus:                    16960.955   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1005795.090
Skew:                           3.304   Prob(JB):                         0.00
Kurtosis:                      35.773   Cond. No.                     1.80e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.8e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
add_R_2('Grade', 0.535)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535


Are non-dummies has a R-squared of: 0.535 which is butter then the FSM R-squared of: 0.493. Will using dummies of the grade column imporm this any?

### Grade Dummy Model

First we will need to ake are dummies using get_dummies and then make a new data frame to join them together.

In [12]:
dummy_model('grade', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.582
Method:                 Least Squares   F-statistic:                     2730.
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        13:44:56   Log-Likelihood:            -2.9798e+05
No. Observations:               21597   AIC:                         5.960e+05
Df Residuals:                   21585   BIC:                         5.961e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    1.802e+05   2.38e+05      0.758      0.448   -2.86e+05    6.46e+05
sqft_living   157.2485      2.783     56.501      0.000     151.793     162.704
grade_4     -7.527e+04   2.42e+05     -0.311      0.756    -5.5e+05    3.99e+05
grade_5     -8.633e+04   2.38e+05     -0.363      0.717   -5.53e+05     3.8e+05
grade_6     -6.569e+04   2.38e+05     -0.276      0.782   -5.32e+05       4e+05
grade_7     -4.331e+04   2.38e+05     -0.182      0.855   -5.09e+05    4.23e+05
grade_8       1.92e+04   2.38e+05      0.081      0.936   -4.47e+05    4.85e+05
grade_9      1.425e+05   2.38e+05      0.599      0.549   -3.24e+05    6.09e+05
grade_10     3.386e+05   2.38e+05      1.423      0.155   -1.28e+05    8.05e+05
grade_11     6.264e+05   2.38e+05      2.630      0.009     1.6e+05    1.09e+06
grade_12     1.161e+06   2.39e+05      4.849      0.000    6.92e+05    1.63e+06
grade_13     2.354e+06   2.47e+05      9.516      0.000    1.87e+06    2.84e+06
==============================================================================
Omnibus:                    13860.549   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           473194.517
Skew:                           2.575   Prob(JB):                         0.00
Kurtosis:                      25.346   Cond. No.                     1.11e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.11e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
add_R_2('Grade Dummy', 0.582)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582


Are non-dummies R-squared was: 0.535 and the dummies R-squared is: 0.582. 

### Condition Model

Ok lets try another another column and look at condition and see what that does.

In [14]:
df['condition'].value_counts()

3    14020
4     5677
5     1701
2      170
1       29
Name: condition, dtype: int64

Ok looks like the condition column is the same as the grade column. They are both categorical columns, so lets do that same thing with is and do a non-dummy model and a dummy model.

In [15]:
model_tests('condition', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                 1.074e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        13:44:56   Log-Likelihood:            -2.9993e+05
No. Observations:               21597   AIC:                         5.999e+05
Df Residuals:                   21594   BIC:                         5.999e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -1.982e+05   1.05e+04    -18.886      0.000   -2.19e+05   -1.78e+05
sqft_living   282.7197      1.931    146.394      0.000     278.934     286.505
condition    4.408e+04   2725.518     16.173      0.000    3.87e+04    4.94e+04
==============================================================================
Omnibus:                    14831.537   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           553385.017
Skew:                           2.822   Prob(JB):                         0.00
Kurtosis:                      27.148   Cond. No.                     1.39e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.39e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
add_R_2('Condition', 0.499)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499


Our non-dummy condition model doesn't seem to help us out any when comparing the two R-squared scores: 0.493 FSM vs 0.499 Non-Dummy Conition Model. Lets see if that is the same if we use get_dummies. 

### Condition Dummy Model

In [17]:
dummy_model('condition', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                     4307.
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        13:44:57   Log-Likelihood:            -2.9992e+05
No. Observations:               21597   AIC:                         5.999e+05
Df Residuals:                   21591   BIC:                         5.999e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -1.193e+04   4.83e+04     -0.247      0.805   -1.07e+05    8.28e+04
sqft_living   282.8679      1.941    145.756      0.000     279.064     286.672
condition_2 -6.179e+04   5.22e+04     -1.183      0.237   -1.64e+05    4.06e+04
condition_3 -5.384e+04   4.84e+04     -1.113      0.266   -1.49e+05    4.09e+04
condition_4 -1.868e+04   4.84e+04     -0.386      0.700   -1.14e+05    7.62e+04
condition_5  5.229e+04   4.87e+04      1.074      0.283   -4.32e+04    1.48e+05
==============================================================================
Omnibus:                    14845.154   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           556085.012
Skew:                           2.825   Prob(JB):                         0.00
Kurtosis:                      27.208   Cond. No.                     1.39e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.39e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
add_R_2('Condition Dummy', 0.499)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499


So are dummy and non-dummy condition models both have a R-squared of: 0.499 which isn't much of an imporvement of the FSM R-squared of: 0.493.

### Floors Model

In [19]:
df['floors'].value_counts()

1.0    10673
2.0     8235
1.5     1910
3.0      611
2.5      161
3.5        7
Name: floors, dtype: int64

Looks like the floor column isnt a categorical column like the first two where. Let go ahead and add this column to are model and see what is does.

In [20]:
model_tests('floors', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                 1.049e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        13:44:57   Log-Likelihood:            -3.0006e+05
No. Observations:               21597   AIC:                         6.001e+05
Df Residuals:                   21594   BIC:                         6.002e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -5.089e+04   5784.516     -8.798      0.000   -6.22e+04   -3.96e+04
sqft_living   279.5098      2.074    134.799      0.000     275.446     283.574
floors       6503.9327   3527.492      1.844      0.065    -410.212    1.34e+04
==============================================================================
Omnibus:                    14847.016   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           548710.587
Skew:                           2.830   Prob(JB):                         0.00
Kurtosis:                      27.036   Cond. No.                     8.06e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.06e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
add_R_2('Floors', 0.493)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499
5,Floors,0.493


Our FSM and the floors model both have the same R-squared of: 0.493. I belive that it is safe to say that floors does have an significant effect if any on the sale price.

### Zip Code Model

For our next model let see what effect sip codes will have. As before we start by looking at the values in the zip code column.

In [22]:
df['zipcode'].value_counts()

98103    602
98038    589
98115    583
98052    574
98117    553
        ... 
98102    104
98010    100
98024     80
98148     57
98039     50
Name: zipcode, Length: 70, dtype: int64

Doesn't seem to be anything out of the normal so lets go ahead and put  it into a model and see if we get a better R-squared of: 0.493 or not.

In [23]:
model_tests('zipcode',df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.501
Model:                            OLS   Adj. R-squared:                  0.500
Method:                 Least Squares   F-statistic:                 1.082e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        13:44:57   Log-Likelihood:            -2.9989e+05
No. Observations:               21597   AIC:                         5.998e+05
Df Residuals:                   21594   BIC:                         5.998e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -6.096e+07   3.31e+06    -18.442      0.000   -6.74e+07   -5.45e+07
sqft_living   288.0945      1.964    146.689      0.000     284.245     291.944
zipcode       620.9595     33.695     18.429      0.000     554.914     687.005
==============================================================================
Omnibus:                    14831.494   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           550498.390
Skew:                           2.824   Prob(JB):                         0.00
Kurtosis:                      27.080   Cond. No.                     1.84e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.84e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
add_R_2("Zipcode", 0.501)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499
5,Floors,0.493
6,Zipcode,0.501


OK cool it didn't go up by a whole lot but an but a small increces is better then not changing at all.

In [25]:
dummy_model('zipcode', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.730
Model:                            OLS   Adj. R-squared:                  0.729
Method:                 Least Squares   F-statistic:                     831.2
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        13:44:57   Log-Likelihood:            -2.9325e+05
No. Observations:               21597   AIC:                         5.866e+05
Df Residuals:                   21526   BIC:                         5.872e+05
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -1.993e+05   1.05e+04    -18.977      0.000    -2.2e+05   -1.79e+05
sqft_living     252.3739      1.574    160.382      0.000     249.290     255.458
zipcode_98002  2.275e+04   1.69e+04      1.347      0.178   -1.03e+04    5.59e+04
zipcode_98003  6582.2750   1.52e+04      0.432      0.666   -2.33e+04    3.64e+04
zipcode_98004  8.216e+05   1.48e+04     55.504      0.000    7.93e+05    8.51e+05
zipcode_98005  3.391e+05   1.79e+04     18.944      0.000    3.04e+05    3.74e+05
zipcode_98006  3.303e+05   1.33e+04     24.818      0.000    3.04e+05    3.56e+05
zipcode_98007  2.658e+05    1.9e+04     13.995      0.000    2.29e+05    3.03e+05
zipcode_98008  3.065e+05   1.52e+04     20.181      0.000    2.77e+05    3.36e+05
zipcode_98010  8.346e+04   2.16e+04      3.862      0.000    4.11e+04    1.26e+05
zipcode_98011   1.21e+05    1.7e+04      7.118      0.000    8.77e+04    1.54e+05
zipcode_98014  1.204e+05   1.99e+04      6.047      0.000    8.13e+04    1.59e+05
zipcode_98019  7.604e+04   1.71e+04      4.436      0.000    4.24e+04     1.1e+05
zipcode_98022  5.232e+04   1.61e+04      3.256      0.001    2.08e+04    8.38e+04
zipcode_98023 -1.614e+04   1.32e+04     -1.222      0.222    -4.2e+04    9755.628
zipcode_98024   1.89e+05   2.36e+04      7.996      0.000    1.43e+05    2.35e+05
zipcode_98027  1.817e+05   1.38e+04     13.149      0.000    1.55e+05    2.09e+05
zipcode_98028   1.26e+05   1.52e+04      8.300      0.000    9.63e+04    1.56e+05
zipcode_98029  2.354e+05   1.47e+04     16.036      0.000    2.07e+05    2.64e+05
zipcode_98030   174.7060   1.56e+04      0.011      0.991   -3.05e+04    3.08e+04
zipcode_98031  1.261e+04   1.53e+04      0.822      0.411   -1.75e+04    4.27e+04
zipcode_98032  1.193e+04   1.98e+04      0.601      0.548    -2.7e+04    5.08e+04
zipcode_98033  4.023e+05   1.37e+04     29.457      0.000    3.76e+05    4.29e+05
zipcode_98034  2.216e+05    1.3e+04     17.081      0.000    1.96e+05    2.47e+05
zipcode_98038  2.419e+04   1.28e+04      1.892      0.059    -873.800    4.93e+04
zipcode_98039  1.401e+06    2.9e+04     48.308      0.000    1.34e+06    1.46e+06
zipcode_98040  6.101e+05   1.53e+04     39.836      0.000     5.8e+05     6.4e+05
zipcode_98042  1.072e+04    1.3e+04      0.827      0.408   -1.47e+04    3.61e+04
zipcode_98045   1.17e+05   1.64e+04      7.151      0.000    8.49e+04    1.49e+05
zipcode_98052   2.48e+05   1.29e+04     19.276      0.000    2.23e+05    2.73e+05
zipcode_98053  2.152e+05   1.39e+04     15.479      0.000    1.88e+05    2.42e+05
zipcode_98055  5.313e+04   1.54e+04      3.446      0.001    2.29e+04    8.34e+04
zipcode_98056  1.111e+05   1.38e+04      8.031      0.000     8.4e+04    1.38e+05
zipcode_98058  3.562e+04   1.35e+04      2.642      0.008    9195.312     6.2e+04
zipcode_98059  8.628e+04   1.34e+04      6.430   

In [26]:
add_R_2('Zipcode Dummy', 0.730)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499
5,Floors,0.493
6,Zipcode,0.501
7,Zipcode Dummy,0.730


This one does have a high R-squared but that may just because that is so many parameters that we add for zipcode.

### Waterfront Model

Lets look at the waterfront column and see what we get for this R-squared.

In [27]:
df['waterfront'].value_counts()

0.0    21451
1.0      146
Name: waterfront, dtype: int64

In [28]:
model_tests('waterfront', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.529
Method:                 Least Squares   F-statistic:                 1.215e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        13:44:57   Log-Likelihood:            -2.9925e+05
No. Observations:               21597   AIC:                         5.985e+05
Df Residuals:                   21594   BIC:                         5.985e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -3.303e+04   4255.379     -7.762      0.000   -4.14e+04   -2.47e+04
sqft_living   272.7834      1.878    145.249      0.000     269.102     276.465
waterfront   8.651e+05    2.1e+04     41.115      0.000    8.24e+05    9.06e+05
==============================================================================
Omnibus:                    13414.805   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           434862.300
Skew:                           2.469   Prob(JB):                         0.00
Kurtosis:                      24.421   Cond. No.                     2.79e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.79e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [29]:
add_R_2('Waterfront', 0.530)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499
5,Floors,0.493
6,Zipcode,0.501
7,Zipcode Dummy,0.730
8,Waterfront,0.530


Alright waterfront has a R-squared of 0.530 that no bad at all.

###  View Model

Up next we are going to look at view and see what we get for the R-squared.

In [33]:
df['view'].value_counts()

0.0    19485
2.0      957
3.0      508
1.0      330
4.0      317
Name: view, dtype: int64

In [31]:
model_tests('view', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.534
Model:                            OLS   Adj. R-squared:                  0.534
Method:                 Least Squares   F-statistic:                 1.239e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        13:46:22   Log-Likelihood:            -2.9914e+05
No. Observations:               21597   AIC:                         5.983e+05
Df Residuals:                   21594   BIC:                         5.983e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -1.795e+04   4266.712     -4.206      0.000   -2.63e+04   -9583.294
sqft_living   256.8943      1.937    132.650      0.000     253.098     260.690
view         1.022e+05   2325.214     43.933      0.000    9.76e+04    1.07e+05
==============================================================================
Omnibus:                    14361.628   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           576631.469
Skew:                           2.661   Prob(JB):                         0.00
Kurtosis:                      27.748   Cond. No.                     5.71e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.71e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [32]:
add_R_2('View', 0.534)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499
5,Floors,0.493
6,Zipcode,0.501
7,Zipcode Dummy,0.730
8,Waterfront,0.530
9,View,0.534


The View column in our data frame is the number of times that a house has been view. And is looks like it has a high R-squared sorce.

### Bedrooms and Bathrooms Model

Next up lets look and see what we get with bedrooms and bathrooms.

In [38]:
df['bedrooms'].value_counts()

3     9824
4     6882
2     2760
5     1601
6      272
1      196
7       38
8       13
9        6
10       3
33       1
11       1
Name: bedrooms, dtype: int64

In [43]:
model_tests('bedrooms', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                 1.110e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        13:57:46   Log-Likelihood:            -2.9976e+05
No. Observations:               21597   AIC:                         5.995e+05
Df Residuals:                   21594   BIC:                         5.995e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    8.091e+04   6638.826     12.187      0.000    6.79e+04    9.39e+04
sqft_living   314.6011      2.344    134.232      0.000     310.007     319.195
bedrooms    -5.783e+04   2322.987    -24.896      0.000   -6.24e+04   -5.33e+04
==============================================================================
Omnibus:                    14391.571   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           488612.629
Skew:                           2.728   Prob(JB):                         0.00
Kurtosis:                      25.654   Cond. No.                     8.91e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.91e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [44]:
add_R_2('Bedrooms', 0.507)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499
5,Floors,0.493
6,Zipcode,0.501
7,Zipcode Dummy,0.730
8,Waterfront,0.530
9,View,0.534


In [37]:
df['bathrooms'].value_counts()

2.50    5377
1.00    3851
1.75    3048
2.25    2047
2.00    1930
1.50    1445
2.75    1185
3.00     753
3.50     731
3.25     589
3.75     155
4.00     136
4.50     100
4.25      79
0.75      71
4.75      23
5.00      21
5.25      13
5.50      10
1.25       9
6.00       6
5.75       4
0.50       4
6.25       2
8.00       2
6.75       2
6.50       2
7.75       1
7.50       1
Name: bathrooms, dtype: int64

In [41]:
model_tests('bathrooms', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                 1.049e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        13:56:54   Log-Likelihood:            -3.0006e+05
No. Observations:               21597   AIC:                         6.001e+05
Df Residuals:                   21594   BIC:                         6.002e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -3.992e+04   5237.187     -7.622      0.000   -5.02e+04   -2.97e+04
sqft_living   284.0883      2.962     95.926      0.000     278.284     289.893
bathrooms   -5095.2734   3535.830     -1.441      0.150    -1.2e+04    1835.215
==============================================================================
Omnibus:                    14783.268   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           540478.758
Skew:                           2.815   Prob(JB):                         0.00
Kurtosis:                      26.852   Cond. No.                     7.27e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.27e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [45]:
add_R_2('Bathrooms', 0.493)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499
5,Floors,0.493
6,Zipcode,0.501
7,Zipcode Dummy,0.730
8,Waterfront,0.530
9,View,0.534


Looking at these two R-squared values we can see that bathrooms had no effect on the score and bedrooms did even if it was a small one.

Lets go ahead and finish up the last few columns.

### Square Feet

Now let do the square feet and see what happens.

In [48]:
model_tests('sqft_lot ', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.494
Model:                            OLS   Adj. R-squared:                  0.494
Method:                 Least Squares   F-statistic:                 1.053e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        15:47:52   Log-Likelihood:            -3.0004e+05
No. Observations:               21597   AIC:                         6.001e+05
Df Residuals:                   21594   BIC:                         6.001e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -4.433e+04   4405.869    -10.062      0.000    -5.3e+04   -3.57e+04
sqft_living   283.1438      1.967    143.927      0.000     279.288     287.000
sqft_lot       -0.2915      0.044     -6.684      0.000      -0.377      -0.206
==============================================================================
Omnibus:                    14737.142   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           534322.524
Skew:                           2.805   Prob(JB):                         0.00
Kurtosis:                      26.713   Cond. No.                     1.09e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.09e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [49]:
add_R_2('Sqft Lot', 0.494)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499
5,Floors,0.493
6,Zipcode,0.501
7,Zipcode Dummy,0.730
8,Waterfront,0.530
9,View,0.534


In [51]:
model_tests('sqft_above', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                 1.050e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        15:49:52   Log-Likelihood:            -3.0005e+05
No. Observations:               21597   AIC:                         6.001e+05
Df Residuals:                   21594   BIC:                         6.001e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -4.127e+04   4455.441     -9.262      0.000      -5e+04   -3.25e+04
sqft_living   295.7265      4.026     73.448      0.000     287.835     303.618
sqft_above    -18.8097      4.466     -4.212      0.000     -27.563     -10.056
==============================================================================
Omnibus:                    14762.837   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           539868.557
Skew:                           2.809   Prob(JB):                         0.00
Kurtosis:                      26.841   Cond. No.                     7.49e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.49e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [52]:
add_R_2('Sqft Above', 0.493)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499
5,Floors,0.493
6,Zipcode,0.501
7,Zipcode Dummy,0.730
8,Waterfront,0.530
9,View,0.534


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     21597 non-null  float64
 9   view           21597 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  float64
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   21597 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [54]:
model_tests('sqft_basement', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                 1.051e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        15:52:46   Log-Likelihood:            -3.0005e+05
No. Observations:               21597   AIC:                         6.001e+05
Df Residuals:                   21594   BIC:                         6.001e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -4.106e+04   4453.261     -9.220      0.000   -4.98e+04   -3.23e+04
sqft_living     276.6134      2.146    128.920      0.000     272.408     280.819
sqft_basement    20.6946      4.479      4.620      0.000      11.916      29.474
==============================================================================
Omnibus:                    14754.603   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           538977.524
Skew:                           2.807   Prob(JB):                         0.00
Kurtosis:                      26.821   Cond. No.                     5.75e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.75e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [55]:
add_R_2('Sqft Basement', 0.493)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499
5,Floors,0.493
6,Zipcode,0.501
7,Zipcode Dummy,0.730
8,Waterfront,0.530
9,View,0.534


In [56]:
model_tests('sqft_living15', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.500
Method:                 Least Squares   F-statistic:                 1.078e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        15:55:55   Log-Likelihood:            -2.9991e+05
No. Observations:               21597   AIC:                         5.998e+05
Df Residuals:                   21594   BIC:                         5.999e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -9.926e+04   5427.190    -18.290      0.000    -1.1e+05   -8.86e+04
sqft_living     242.4444      2.945     82.329      0.000     236.672     248.216
sqft_living15    68.0527      3.946     17.248      0.000      60.319      75.786
==============================================================================
Omnibus:                    15568.864   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           682653.257
Skew:                           2.983   Prob(JB):                         0.00
Kurtosis:                      29.889   Cond. No.                     9.42e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.42e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [58]:
add_R_2('Sqft Living15', 0.500)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499
5,Floors,0.493
6,Zipcode,0.501
7,Zipcode Dummy,0.730
8,Waterfront,0.530
9,View,0.534


In [59]:
model_tests('sqft_lot15', df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.495
Model:                            OLS   Adj. R-squared:                  0.495
Method:                 Least Squares   F-statistic:                 1.058e+04
Date:                Wed, 02 Jun 2021   Prob (F-statistic):               0.00
Time:                        15:57:21   Log-Likelihood:            -3.0001e+05
No. Observations:               21597   AIC:                         6.000e+05
Df Residuals:                   21594   BIC:                         6.001e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    -4.31e+04   4401.296     -9.793      0.000   -5.17e+04   -3.45e+04
sqft_living   284.4173      1.969    144.455      0.000     280.558     288.276
sqft_lot15     -0.6490      0.066     -9.793      0.000      -0.779      -0.519
==============================================================================
Omnibus:                    14706.507   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           532440.211
Skew:                           2.796   Prob(JB):                         0.00
Kurtosis:                      26.673   Cond. No.                     7.46e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.46e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [60]:
add_R_2('Sqft Lot15', 0.495)

,Name,R-squared
0,FSM,0.493
1,Grade,0.535
2,Grade Dummy,0.582
3,Condition,0.499
4,Condition Dummy,0.499
5,Floors,0.493
6,Zipcode,0.501
7,Zipcode Dummy,0.730
8,Waterfront,0.530
9,View,0.534


In [62]:
df.corr()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,1.000000,-0.016772,0.001150,0.005162,-0.012241,-0.131911,0.018608,-0.003599,0.011772,-0.023803,0.008188,-0.010799,-0.004359,0.021617,-0.010612,-0.008211,-0.001798,0.020672,-0.002701,-0.138557
price,-0.016772,1.000000,0.308787,0.525906,0.701917,0.089876,0.256804,0.264306,0.393497,0.036056,0.667951,0.605368,0.321108,0.053953,0.117855,-0.053402,0.306692,0.022036,0.585241,0.082845
bedrooms,0.001150,0.308787,1.000000,0.514508,0.578212,0.032471,0.177944,-0.002127,0.078354,0.026496,0.356563,0.479386,0.297229,0.155670,0.017900,-0.154092,-0.009951,0.132054,0.393406,0.030690
bathrooms,0.005162,0.525906,0.514508,1.000000,0.755758,0.088373,0.502582,0.063629,0.186016,-0.126479,0.665838,0.686668,0.278485,0.507173,0.047177,-0.204786,0.024280,0.224903,0.569884,0.088303
sqft_living,-0.012241,0.701917,0.578212,0.755758,1.000000,0.173453,0.353953,0.104637,0.281715,-0.059445,0.762779,0.876448,0.428660,0.318152,0.051060,-0.199802,0.052155,0.241214,0.756402,0.184342
sqft_lot,-0.131911,0.089876,0.032471,0.088373,0.173453,1.000000,-0.004814,0.021459,0.075054,-0.008830,0.114731,0.184139,0.015031,0.052946,0.004979,-0.129586,-0.085514,0.230227,0.144763,0.718204
floors,0.018608,0.256804,0.177944,0.502582,0.353953,-0.004814,1.000000,0.020797,0.028414,-0.264075,0.458794,0.523989,-0.241866,0.489193,0.003793,-0.059541,0.049239,0.125943,0.280102,-0.010722
waterfront,-0.003599,0.264306,-0.002127,0.063629,0.104637,0.021459,0.020797,1.000000,0.380543,0.016648,0.082818,0.071778,0.083050,-0.024487,0.073939,0.028923,-0.012157,-0.037628,0.083823,0.030658
view,0.011772,0.393497,0.078354,0.186016,0.281715,0.075054,0.028414,0.380543,1.000000,0.045622,0.249082,0.166017,0.270629,-0.054452,0.090339,0.085059,0.006321,-0.077702,0.278928,0.073083
condition,-0.023803,0.036056,0.026496,-0.126479,-0.059445,-0.008830,-0.264075,0.016648,0.045622,1.000000,-0.146896,-0.158904,0.168482,-0.361592,-0.055808,0.002888,-0.015102,-0.105877,-0.093072,-0.003126
